In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
# n = len(documents)
# for i in range(n):
#     documents[i]['id']=1

In [4]:

import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [5]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [6]:
from collections import defaultdict

In [7]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [8]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

593f7569 2


In [9]:
hashes['593f7569']

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [10]:
import json
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)



In [11]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [12]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [13]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)


In [15]:
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

I'm happy to help you with that! However, I need more information about the course you're referring to. Can you please provide more details such as:

1. What is the name of the course?
2. Who is offering the course?
3. What is the format of the course (online, in-person, etc.)?
4. How much time has passed since the course started?

With more information, I can better assess the likelihood of you being able to join the course and provide more accurate guidance.


In [16]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
    )

    json_response = response.choices[0].message.content
    return json_response

In [17]:

from tqdm.auto import tqdm

In [18]:
results = {}

In [19]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|          | 0/948 [00:00<?, ?it/s]

In [30]:
results

{'c02e79ef': 'Here are 5 questions the student might ask based on the FAQ record:\n\n[\n"When does the course start exactly?",\n"When will the first live session of the course take place?",\n"How do I know when the course is about to start?",\n"What is the recommended way to stay updated on course-related announcements?",\n"Can I register for the course after it has started?"\n]',
 '1f6520ca': 'Here are 5 questions a student might ask based on the FAQ record:\n\n[\n"Can I take the data engineering zoom camp course without any prior experience?",\n"What are the necessary skills or knowledge required to take this data engineering course?",\n"I have a background in computer science, but not in data engineering. Am I qualified to take this course?",\n"What do I need to learn or review before starting the data engineering zoom camp course?",\n"Is there a specific previous course or training I must have completed to enroll in data engineering zoom camp?"\n]',
 '7842b56a': 'Here are 5 questio

In [20]:
import pickle

In [24]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

FileNotFoundError: [Errno 2] No such file or directory: 'results'

In [22]:
results['1f6520ca']

'Here are 5 questions a student might ask based on the FAQ record:\n\n[\n"Can I take the data engineering zoom camp course without any prior experience?",\n"What are the necessary skills or knowledge required to take this data engineering course?",\n"I have a background in computer science, but not in data engineering. Am I qualified to take this course?",\n"What do I need to learn or review before starting the data engineering zoom camp course?",\n"Is there a specific previous course or training I must have completed to enroll in data engineering zoom camp?"\n]'

In [23]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    parsed_resulst[doc_id] = json.loads(json_questions)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [25]:
doc_index = {d['id']: d for d in documents}

In [26]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [27]:
import pandas as pd

In [28]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [29]:
df

,question,course,document
